In [1]:
from pyspark import SparkConf,SparkContext
conf=SparkConf().setMaster("local").setAppName("AudioRecommender")
sc=SparkContext(conf=conf)
from random import randrange
from operator import itemgetter
from pyspark.mllib.recommendation import ALS, Rating

In [2]:
base="data/"
rawUserArtistData = sc.textFile(base + "user_artist_data.txt").cache()
rawArtistData = sc.textFile(base + "artist_data.txt").cache()
rawArtistAlias = sc.textFile(base + "artist_alias.txt").cache()

In [3]:
def _parseArtistAlias(line):
    fields = line.split('\t')
    try:
        artist = int(fields[0])
        alias = int(fields[1])
        return (artist,alias)
    except:
        return (None,None)
artistAlias=rawArtistAlias.map(_parseArtistAlias).collectAsMap()

In [4]:
def _parseUserArtist(line):
    fields = line.split(' ')
    user = int(fields[0])
    artist = int(fields[1])
    return (user,artist)
UserArtistData = rawUserArtistData.map(_parseUserArtist)

In [5]:
def _parseArtistData(line):
    fields = line.split('\t')
    try:
        artist_id = int(fields[0])#id is reserved in Python
        name = str(fields[1].strip())
        return (artist_id,name)
    except:
        return (None,None)
ArtistData=rawArtistData.map(_parseArtistData)  

In [6]:
#rawUserArtistData.map(lambda line: int(line.split(' ')[0])).stats()

In [7]:
#rawUserArtistData.map(lambda line: int(line.split(' ')[1])).stats()

In [8]:
def buildCounts(rawUserArtistData, bArtistAlias):
    def getArtistRating(line):
        (userID, artistID, count) = map(lambda x: int(x), line.split(' '))
        try:
            finalArtistID = bArtistAlias.value[artistID]
        except KeyError:
            finalArtistID = artistID
        return Rating(userID, finalArtistID, count)

    return rawUserArtistData.map(lambda line: getArtistRating(line))
bArtistAlias = sc.broadcast(artistAlias)
trainData=buildCounts(rawUserArtistData,bArtistAlias)
trainData.cache()

PythonRDD[7] at RDD at PythonRDD.scala:48

In [9]:
model=ALS.trainImplicit(ratings=trainData,rank=10,iterations=5,lambda_=0.01,alpha=1.0)

In [14]:
userID=2093760
recommendations=model.recommendProducts(userID,10)
recommendedProductIDs = map(lambda rec: rec.product, recommendations)

In [18]:
stack=[]
for i in recommendedProductIDs:
    stack.append(i)

In [16]:
recommendedArtists = ArtistData.filter(lambda artist: artist[0] in stack).collect()

In [17]:
recommendedArtists

[(1004028, 'Notorious B.I.G.'),
 (2814, '50 Cent'),
 (4605, 'Snoop Dogg'),
 (829, 'Nas'),
 (1007614, 'Jay-Z'),
 (1037970, 'Kanye West'),
 (1811, 'Dr. Dre'),
 (1003249, 'Ludacris'),
 (1001819, '2Pac'),
 (1300642, 'The Game')]